In [1]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import urllib.parse
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.cluster import KMeans   
from sklearn import datasets
from pycaret.clustering import *
from pycaret.datasets import *


In [2]:

logtoparse=''
badwordshttprequest=['sleep','uid', 'eid','select','waitfor','from','system','select',
					 'union','order by','group by','where','alert','script','sleep','convert']

def Extratedfeature(path):
	path=urllib.parse.unquote(path)
	badwords_count=0
	# classy='bad'
	Nsingle_q=path.count("'") 
	Ndouble_q=path.count("\"")
	Ndashes=path.count("--")
	Nbraces=path.count("(")
	Nspaces=path.count(' ')
	classy=''
	for i in badwordshttprequest:
			badwords_count+=path.count(i)
		
		
	# for i in badwordshttprequest:
			# Nsingle_q=body.count("'")
			# Ndouble_q=body.count("\"")
			# Ndashes=body.count("--")
			# Nbraces=body.count("(")
			# Nspaces=body.count(' ')
			
			# for k,v in header.iterintems():
			# 	st=str()
			# badwords_count+=path.count(i)
	lst=[Nsingle_q,Ndouble_q,Ndashes,Nbraces,Nspaces,badwords_count,classy]
	print(lst)
	return pd.DataFrame([lst],columns=['Nsingle_q','Ndouble_q','Ndashes','Nspaces','Nbraces','badwords_count','class'])

In [3]:
df=pd.read_csv('finaldataset161023.csv')
newdf=df.dropna(inplace=False)
# newdf.to_csv('finaldataset161023cleaned.csv')
keyonly=newdf[['Nsingle_q','Ndouble_q','Ndashes','Nspaces','Nbraces','badwords_count']]

clu1=setup(data=newdf, normalize=True, categorical_features=['class'],numeric_features=['Nsingle_q','Ndouble_q','Ndashes','Nspaces','Nbraces','badwords_count'],ignore_features=['method','body','path','header'])
# x=np.array(keyonly)

# wcss=[]
# K=range(1,10)
# for k in K:  
#  kmeanModel = KMeans(n_clusters=k)  
#  kmeanModel.fit(x)  
#  wcss.append(kmeanModel.inertia_) # Plotting the distortions  
 
# plt.figure(figsize=(16,8))
# plt.plot(K, wcss)
# plt.xlabel('k')
# plt.ylabel('wcss')  
# plt.title('The Elbow Method showing the optimal clusters')  
# plt.show()




,Description,Value
0,Session id,6383
1,Original data shape,"(4391, 11)"
2,Transformed data shape,"(4391, 7)"
3,Ignore features,4
4,Ordinal features,1
5,Numeric features,6
6,Categorical features,1
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [4]:
# kmeans_model = KMeans(n_clusters=3, random_state=32932,max_iter=300)
# kmeans_predict = kmeans_model.fit_predict(x)
kmeans=clu1.create_model('kmeans',num_clusters=2)
kmeans


KMeans(n_clusters=2, random_state=6383)

In [8]:
# col1=label_0.columns
# col2=label_1.columns

# plt.scatter(label_0[col1[0]],label_0[col1[1]], color='red')
# plt.scatter(label_1[col2[0]],label_1[col2[1]], color='black')
# plt.show()
# keyonly['Cluster']=kmeans_predict

#creation of the proxy server
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes={}

    @classmethod
    def set_routes(cls,proxy_routes):
        cls.proxy_routes=proxy_routes
    
    def do_GET(self):
        part1=self.path.split('/')
        parts=self.path
        print(parts)
        live_data=Extratedfeature(parts)
        result=predict_model(kmeans,data=live_data)
        print(result)

        if result['Cluster'][0]=='Cluster 1':
            print('Intrusion Detected')

        if len(part1) >=2:

            self.proxy_request('http://'+part1[2]+'/')


        else :
            super().do_GET()
    
    def proxy_request(self,url):

        try:
            response=request.urlopen(url)
        except error.HTTPError as e:
            print('err')
            self.end_headers()
            return
        
        self.send_response_only(response)

        for name, value in response:
            self.send_head(name,value)
        self.end_headers()
        self.copyfile(response, self.wfile)

    

SimpleHTTPProxy.set_routes({'proxy_route':'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1',8080),SimpleHTTPProxy) as httpd:
    host, port=httpd.socket.getsockname()
    print(f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print('\nKeyboard interrupt received, existing')
        exit(0)
        


        


Listening on http://127.0.0.1:8080


127.0.0.1 - - [18/Oct/2023 10:04:36] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Oct/2023 10:04:36] "CONNECT play.google.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [18/Oct/2023 10:04:45] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [18/Oct/2023 10:04:45] "CONNECT www.bing.com:443 HTTP/1.1" 501 -


http://demo.testfire.net/search.jsp?query=%27User-Agent%27%3A+%27+Mozilla%2F5.0+%28Windows+NT+6.1%3B+WOW64%29+AppleWebKit%2F537.21+%28KHTML%2C+like+Gecko%29+Chrome%2F41.0.2228.0+Safari%2F537.21%27%2C+%27X-Forwarded-Host%27%3A+%27+%28select%280%29from%28select%28sleep%2812%29%29%29v%29%2F*%5C%5C%5C%27%2B%28select%280%29from%28select%28sleep%2812%29%29%29v%29%2B%5C%5C%5C%27%22%22%2B%28select%280%29from%28select%28sleep%2812%29%29%29v%29%2B%22%22*%2F%27%2C+%27X-Requested-With%27%3A+%27+XMLHttpRequest%27%2C+%27Referer%27%3A+%27+http%3A%2F%2Fdemo.testfire.net%27%2C+%27Cookie%27%3A+%27+JSESSIONID%3D1789B1649DBE83467C45C05E5E1DB197%27%2C+%27Host%27%3A+%27+demo.testfire.net%27%2C+%27Connection%27%3A+%27+close%27%2C+%27Accept-Encoding%27%3A+%27+gzip%2C+deflate%2C+br%27%2C+%27Acunetix-Product%27%3A+%27+WVS%2F10.0+%28Acunetix+Web+Vulnerability+Scanner+-+Free+Edition%29%27%2C+%27Acunetix-Scanning-agreement%27%3A+%27+Third+Party+Scanning+PROHIBITED%27%2C+%27Acunetix-User-agreement%27%3A+%27+http%3A

In [6]:
# plt.scatter(x[kmeans_predict == 0, 0], x[kmeans_predict == 0, 1], s = 100, c = 'red')  
# plt.scatter(x[kmeans_predict == 1, 0], x[kmeans_predict == 1, 1], s = 100, c = 'blue')  
# plt.scatter(x[kmeans_predict == 2, 0], x[kmeans_predict == 2, 1], s = 100, c = 'green')# Plott
# ing the centroids of the clusters  
# plt.scatter(kmeans_model.cluster_centers_[:, 0], kmeans_model.cluster_centers_[:,1], s = 100, c = 'yellow')
# plt.show()
# plot_model(kmeans)
evaluate_model(kmeans)



interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [6]:
kmeans_result=assign_model(kmeans)
kmeans_result.to_csv(r'kmeansresult.csv')

In [6]:
dff=pd.read_csv('finaldataset161023cleaned.csv')
dff.head(10)

,Unnamed: 0,method,header,body,path,Nsingle_q,Ndouble_q,Ndashes,Nbraces,Nspaces,badwords_count,class
0,0,GET,"{'Host': ' demo.testfire.net', 'Connection': '...",/,HTTP/1.1,0,0,0,2,0,0,good
1,1,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/VHZo2EZh,HTTP/1.1,0,0,0,2,0,0,good
2,2,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/KYqIcNEl,HTTP/1.1,0,0,0,2,0,0,good
3,3,GET,{'Cookie': ' JSESSIONID=6A672F0C0562B4FA031BF6...,/B4XTfL7l,HTTP/1.1,0,0,0,2,0,0,good
4,4,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/,HTTP/1.1,0,0,0,3,0,0,good
5,5,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/search.jsp?query=1,HTTP/1.1,0,0,0,3,0,2,good
6,6,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=business_retirement.htm,HTTP/1.1,0,0,0,3,0,2,good
7,7,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=business_deposit.htm,HTTP/1.1,0,0,0,3,0,2,good
8,8,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp?content=inside.htm,HTTP/1.1,0,0,0,3,0,2,good
9,9,GET,"{'Pragma': ' no-cache', 'Cache-Control': ' no-...",/index.jsp,HTTP/1.1,0,0,0,3,0,2,good


In [9]:
k="'User-Agent': ' Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.21 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.21', 'X-Forwarded-Host': ' (select(0)from(select(sleep(12)))v)/*\\\'+(select(0)from(select(sleep(12)))v)+\\\'""+(select(0)from(select(sleep(12)))v)+""*/', 'X-Requested-With': ' XMLHttpRequest', 'Referer': ' http://demo.testfire.net', 'Cookie': ' JSESSIONID=1789B1649DBE83467C45C05E5E1DB197', 'Host': ' demo.testfire.net', 'Connection': ' close', 'Accept-Encoding': ' gzip, deflate, br', 'Acunetix-Product': ' WVS/10.0 (Acunetix Web Vulnerability Scanner - Free Edition)', 'Acunetix-Scanning-agreement': ' Third Party Scanning PROHIBITED', 'Acunetix-User-agreement': ' http://www.acunetix.com/wvs/disc.htm', 'Accept': ' */*''"
live_data=Extratedfeature(k)

resulttest=predict_model(kmeans,data=live_data)
# resulttest.to_csv(r'resultest.csv')
resulttest

if resulttest['Cluster'][0]== 'Cluster 0':
    print('Clean')
else:
    print ('Malicious')


[51, 0, 0, 18, 57, 21, '']
Malicious
